In [1]:
from selenium import webdriver
import json
import time

In [2]:
driver = webdriver.Chrome()


In [3]:
#url = "https://sf-item.taobao.com/sf_item/592140469999.htm?spm=a213w.7398504.paiList.1.a1da26cdBOcTJJ"

#url = "https://sf-item.taobao.com/sf_item/591732235950.htm?spm=a213w.7398504.paiList.6.7ede26cdIZuymq"

url = "https://sf.taobao.com/item_list.htm?spm=a213w.7398504.filter.61.22f526cdj7uTkT&category=50025969" \
    "&auction_source=0&province=%CC%EC%BD%F2&sorder=2&st_param=-1&auction_start_seg=-1"

In [4]:
driver.get(url)

In [5]:
urlList = []

In [6]:
for i in range(2,3):
    driver.find_element_by_link_text(str(i)).click()
    time.sleep(1)
    data = driver.find_element_by_id("sf-item-list-data").get_attribute('innerHTML')
    jsonData =  json.loads(data)
    jd = jsonData["data"]
    for i in range(len(jd)):
        num = jd[i]["id"]
        url = "#pai-item-"+str(num)+" .pic"
        urlList.append(url)
        driver.find_element_by_css_selector(url).click()
        time.sleep(1)
        handles = driver.window_handles
        print(handles)
        driver.switch_to.window(handles[1])
        data = driver.find_element_by_class_name("bid-fail").text
        print(str(i) + data+driver.title)
        driver.close()
        driver.switch_to.window(handles[0])


['CDwindow-3A4959BF0FAAC1ED1934641EF0CA53DE', 'CDwindow-EB396BA5AC30AD2358BC4EAFB4E8D625']
0本场拍卖已结束 ！天津市滨海新区中新天津生态城和旭路1060号季景园9-101室 - 司法拍卖 - 阿里拍卖
['CDwindow-3A4959BF0FAAC1ED1934641EF0CA53DE', 'CDwindow-A9E3C5944DBD2956E8F5E4C766243641']
1本场拍卖已流拍 ！天津市北辰区引河南里1-4-302号房地产 - 司法拍卖 - 阿里拍卖
['CDwindow-3A4959BF0FAAC1ED1934641EF0CA53DE', 'CDwindow-1B600D45BD10520A9E244FFE13D519B9']
2本场拍卖已结束 ！天津市蓟县碧玉家园北侧，福地花园南侧山水颐城北园4-1-201房地产 - 司法拍卖 - 阿里拍卖
['CDwindow-3A4959BF0FAAC1ED1934641EF0CA53DE', 'CDwindow-54872A988FC2D272DFFB50368392772F']
3本场拍卖已流拍 ！天津市蓟县碧玉家园北侧，福地花园南侧山水颐城北园20-5-801房地产 - 司法拍卖 - 阿里拍卖
['CDwindow-3A4959BF0FAAC1ED1934641EF0CA53DE', 'CDwindow-D06A4DB7DD6849C3B9FA31F5B5F3B68B']
4本场拍卖已流拍 ！天津市滨海新区大港世纪大道88号1506号房地产 - 司法拍卖 - 阿里拍卖
['CDwindow-3A4959BF0FAAC1ED1934641EF0CA53DE', 'CDwindow-42BDB6CD69F770117754C55EB96574F4']
5本场拍卖已结束 ！红桥区纪念馆路延长线与本溪路交口东北侧美居花园8-1-18018-1-1801号 - 司法拍卖 - 阿里拍卖
['CDwindow-3A4959BF0FAAC1ED1934641EF0CA53DE', 'CDwindow-F07E6476817250779AD3F3944481F8A2']
6本场拍卖已结束 ！天津市蓟县碧玉家园北侧，福地花园南侧

In [7]:
#for i in range(len(urlList))


#     driver.switchTo().window("vars.get("root").toString()");
#     driver.findElement(By.cssSelector("#pai-item-591508706185 .pic")).click();
#     driver.switchTo().window("vars.get("win8140").toString()");
#     driver.close();

In [8]:
urlList

['#pai-item-590871909472 .pic',
 '#pai-item-592296215593 .pic',
 '#pai-item-592218695978 .pic',
 '#pai-item-592218487814 .pic',
 '#pai-item-592152387339 .pic',
 '#pai-item-592054657921 .pic',
 '#pai-item-592048830017 .pic',
 '#pai-item-592048386042 .pic',
 '#pai-item-592047962769 .pic',
 '#pai-item-591980558218 .pic',
 '#pai-item-591979738960 .pic',
 '#pai-item-591976885596 .pic',
 '#pai-item-591972669484 .pic',
 '#pai-item-591895081218 .pic',
 '#pai-item-591829765445 .pic',
 '#pai-item-591829713499 .pic',
 '#pai-item-591829045033 .pic',
 '#pai-item-591827145465 .pic',
 '#pai-item-591605604210 .pic',
 '#pai-item-591024982061 .pic',
 '#pai-item-590872593331 .pic',
 '#pai-item-590570345848 .pic',
 '#pai-item-591913335846 .pic',
 '#pai-item-591870493079 .pic',
 '#pai-item-590555428093 .pic',
 '#pai-item-591972533660 .pic',
 '#pai-item-590647600859 .pic',
 '#pai-item-590649084235 .pic',
 '#pai-item-592206758561 .pic',
 '#pai-item-592194163067 .pic',
 '#pai-item-591583798643 .pic',
 '#pai-i

In [9]:
#590871909472
#pai-item-591508706185 .pic

In [10]:
#data = driver.find_element_by_class_name("bid-fail").text
#print(data)

In [11]:
#d = driver.find_element_by_id("sf-item-list-data").get_attribute('innerHTML')


In [12]:
#jd = json.loads(d)


In [13]:
# for i in range(len(jd["data"])):
#     print(jd["data"][i]["id"])

In [14]:
#driver.find_element_by_css_selector(url).click()